Soccer wizard

In [1]:
import requests
import pandas as pd
import helper_functions as sc

In [2]:
# get API key
key = open('data/rapid_api_key.txt', 'r').read()

In [3]:
# get leagues
leagues = sc.request_leagues()
for l in leagues:
    if leagues[l]['country'] == 'Germany':
        print('name = ', leagues[l]['name'], 'year = ', leagues[l]['season'], 'key =', leagues[l]['league_id'])

name =  Bundesliga 1 year =  2018 key = 8
name =  Bundesliga 2 year =  2018 key = 9
name =  Bundesliga 1 year =  2017 key = 35
name =  Bundesliga 2 year =  2017 key = 36
name =  Bundesliga 1 year =  2016 key = 54
name =  Bundesliga 2 year =  2016 key = 55
name =  Liga 3 year =  2018 key = 159
name =  Liga 3 year =  2017 key = 160
name =  Liga 3 year =  2016 key = 161
name =  DFB Pokal year =  2016 key = 202
name =  DFB Pokal year =  2017 key = 203
name =  DFB Pokal year =  2018 key = 204
name =  Women Bundesliga year =  2016 key = 226
name =  Women Bundesliga year =  2017 key = 227
name =  Women Bundesliga year =  2018 key = 228


In [4]:
# load data
fixtures = sc.request_data(league=8, fixtures_teams='fixtures')
teams_info = sc.request_data(league=8, fixtures_teams='teams')

In [5]:
# get teams
teams = pd.DataFrame({'id': [], 'team': []})
for num in teams_info:
    teams = teams.append(pd.DataFrame({'id':[teams_info[str(num)]['team_id']], 'team': [teams_info[str(num)]['name']]}))
teams = teams.reset_index(drop=True)
teams.head()

,id,team
0,157,Bayern Munich
1,167,1899 Hoffenheim
2,158,Fortuna Dusseldorf
3,170,FC Augsburg
4,159,Hertha Berlin


In [6]:
# get results
results = pd.DataFrame()
for f in fixtures:
    results = results.append(pd.DataFrame({'round': [fixtures[f]['round']],
                                           'home': [fixtures[f]['homeTeam_id']],
                                           'away': [fixtures[f]['awayTeam_id']],
                                           'goals_home': [fixtures[f]['goalsHomeTeam']],
                                           'goals_away': [fixtures[f]['goalsAwayTeam']],
                                           'match_id': fixtures[f]['fixture_id']}))
results = results.reset_index(drop=True)

# get points
results['points_home'] = results.apply(lambda x: sc.points(x['goals_home'], x['goals_away']), axis=1)
results['points_away'] = results.apply(lambda x: sc.points(x['goals_away'], x['goals_home']), axis=1)
results.head()

,round,home,away,goals_home,goals_away,match_id,points_home,points_away
0,Regular Season - 1,157,167,3,1,2517,3,0
1,Regular Season - 1,158,170,1,2,2518,0,3
2,Regular Season - 1,159,171,1,0,2519,3,0
3,Regular Season - 1,160,169,0,2,2520,0,3
4,Regular Season - 1,161,174,2,1,2521,3,0


In [7]:
# attention ! a lot of API calls!
stats = pd.read_csv('data/stats_2018.csv', index_col=0)
stats.head()

#stats = pd.DataFrame()
#for id in results['match_id']:
#    stat = pd.DataFrame(sc.request_match_statistics(id))
#    stat['match_id'] = id
#    stats = stats.append(stat)
#stats = stats.rename_axis('home_away').reset_index()
#stats.to_csv('data/stats_2018.csv')
#stats.head()


,home_away,Shots on Goal,Shots off Goal,Total Shots,Blocked Shots,Shots insidebox,Shots outsidebox,Fouls,Corner Kicks,Offsides,Ball Possession,Yellow Cards,Red Cards,Goalkeeper Saves,Total passes,Passes accurate,Passes %,match_id
0,away,5,1,8,2,2,6,21,2,2.0,33%,4.0,0.0,2.0,277,208,75%,2517
1,home,5,5,16,6,9,7,5,7,4.0,67%,1.0,0.0,4.0,559,481,86%,2517
2,away,7,4,14,3,9,5,11,5,3.0,49%,0.0,0.0,4.0,422,322,76%,2518
3,home,5,2,8,1,4,4,12,7,1.0,51%,1.0,0.0,4.0,447,348,78%,2518
4,away,3,6,11,2,5,6,10,4,NaN,51%,2.0,0.0,2.0,468,364,78%,2519


In [8]:
rounds = results['round'].unique()
table = {r:{t:{'points':[], 'goals_shot':[], 'goals_received':[]} for t in teams['id']} for r in rounds}

In [20]:
# setup table base results
# make 1 out of two = goal difference and goals_shot (only count goals shot per gameday add substract for difference)
last_round = ''
for r in rounds:
    #print(r)
    for t in teams['id']:
        #print(t)
        try:
            match_id = int(results[(results['round'] == r) & (results['home'] == str(t))]['match_id'])
            stat = stats[(stats['match_id'] == match_id) & (stats['home_away'] == 'home')]
            points = int(results[(results['round'] == r) & (results['home'] == str(t))]['points_home'])
            goals_shot = int(results[(results['round'] == r) & (results['home'] == str(t))]['goals_home'])
            goals_received = int(results[(results['round'] == r) & (results['home'] == str(t))]['goals_away'])
        except:
            match_id = int(results[(results['round'] == r) & (results['away'] == str(t))]['match_id'])
            stat = stats[(stats['match_id'] == match_id) & (stats['home_away'] == 'away')]
            points = int(results[(results['round'] == r) & (results['away'] == str(t))]['points_away'])
            goals_shot = int(results[(results['round'] == r) & (results['away'] == str(t))]['goals_away'])
            goals_received = int(results[(results['round'] == r) & (results['away'] == str(t))]['goals_home'])
        #print(match_id)
        #print(stat)
        table[r][str(t)]['points'] = points
        table[r][str(t)]['posession'] = int(stat['Ball Possession'].values[0][:-1])/100
        table[r][str(t)]['pass_acc'] = int(stat['Passes %'].values[0][:-1])/100
        table[r][str(t)]['shots'] = int(stat['Total Shots'].values[0])
        table[r][str(t)]['shot_acc'] = round(goals_shot/int(stat['Total Shots'].values[0]),2)
        if last_round == '':
            table[r][str(t)]['points_cum'] = points
            table[r][str(t)]['goals_shot'] = goals_shot
            table[r][str(t)]['goals_received'] = goals_received
        else:
            table[r][str(t)]['points_cum'] = table[last_round][str(t)]['points_cum'] + points
            table[r][str(t)]['goals_shot'] = table[last_round][str(t)]['goals_shot'] + goals_shot
            table[r][str(t)]['goals_received'] = table[last_round][str(t)]['goals_received'] + goals_received
        table[r][str(t)]['goal_difference'] = table[r][str(t)]['goals_shot']-table[r][str(t)]['goals_received']
    last_round = r
#table

In [22]:
# get rank & rating
for r in rounds:
    gameday = pd.DataFrame()
    for t in teams['id']:
        gameday = gameday.append(pd.DataFrame({'round': [r],
                                               'team': [t],
                                               'points_cum': table[r][str(t)]['points_cum'],
                                               'goal_difference': table[r][str(t)]['goal_difference']}))
    gameday = gameday.sort_values(['points_cum', 'goal_difference'], ascending = False)
    gameday['rank'] = [i for i in range(1,len(gameday)+1)]
    for t in teams['id']:
        table[r][str(t)]['rank'] = int(gameday[gameday['team'] == t]['rank'])
        table[r][str(t)]['rating'] = sc.get_rating(int(table[r][str(t)]['rank']))

In [23]:
# get form
for t in teams['id']:
    form = []
    form_rating = []
    for r in rounds:
        if len(form) > 4:
            form.pop(0)
            form_rating.pop(0)
        form.append(table[r][t]['points'])
        form_rating.append(table[r][t]['rating'])
        table[r][t]['form'] = form.copy()
        table[r][t]['form_rating'] = form_rating.copy()

In [24]:
# get form statistics
for t in teams['id']:
    form_possession = [] #shot_acc
    form_pass_acc = []
    form_shots = []
    form_shot_acc = [] 
    for r in rounds:
        if len(form_possession) > 4:
            form_possession.pop(0)
            form_pass_acc.pop(0)
            form_shot_acc.pop(0)
            form_shots.pop(0)
        form_possession.append(table[r][t]['posession'])
        form_pass_acc.append(table[r][t]['pass_acc'])
        form_shots.append(table[r][t]['shots'])
        form_shot_acc.append(table[r][t]['shot_acc'])
        table[r][t]['form_possession'] = form_possession.copy()
        table[r][t]['form_pass_acc'] = form_pass_acc.copy()
        table[r][t]['form_shots'] = form_shots.copy()
        table[r][t]['form_shot_acc'] = form_shot_acc.copy()
table

{'Regular Season - 1': {'157': {'points': 3,
   'goals_shot': 3,
   'goals_received': 1,
   'posession': 0.67,
   'pass_acc': 0.86,
   'shots': 16,
   'shot_acc': 0.19,
   'points_cum': 3,
   'goal_difference': 2,
   'rank': 2,
   'rating': 'A',
   'form': [3],
   'form_rating': ['A'],
   'form_possession': [0.67],
   'form_pass_acc': [0.86],
   'form_shots': [16],
   'form_shot_acc': [0.19]},
  '167': {'points': 0,
   'goals_shot': 1,
   'goals_received': 3,
   'posession': 0.33,
   'pass_acc': 0.75,
   'shots': 8,
   'shot_acc': 0.12,
   'points_cum': 0,
   'goal_difference': -2,
   'rank': 15,
   'rating': 'C',
   'form': [0],
   'form_rating': ['C'],
   'form_possession': [0.33],
   'form_pass_acc': [0.75],
   'form_shots': [8],
   'form_shot_acc': [0.12]},
  '158': {'points': 0,
   'goals_shot': 1,
   'goals_received': 2,
   'posession': 0.51,
   'pass_acc': 0.78,
   'shots': 8,
   'shot_acc': 0.12,
   'points_cum': 0,
   'goal_difference': -1,
   'rank': 11,
   'rating': 'C',
   

## Get dataframe with features+target:

In [25]:
data = pd.DataFrame()
for i in range(len(results)):
    team_1 = results.loc[i]['home']
    team_2 = results.loc[i]['away']
    gameday = results.loc[i]['round']
    if gameday in ['Bundesliga - 1', 'Regular Season - 1']:
        last_gameday = gameday
        next
    data = data.append(sc.get_sample_row(gameday, last_gameday, team_1, team_2, results, i, table))
    data = data.append(sc.get_sample_row(gameday, last_gameday, team_2, team_1, results, i, table))
    last_gameday = gameday
data = data.reset_index(drop=True)
data.head()

,round,team_1,goal_difference_1,rating_1,form_1,form_weighted_1,form_possession_1,form_pass_acc_1,form_shot_acc_1,team_2,goal_difference_2,rating_2,form_2,form_weighted_2,form_possession_2,form_pass_acc_2,form_shot_acc_2,home_away,target
0,Regular Season - 1,157,2,A,3.0,12.0,0.67,0.86,0.19,167,-2,C,0.0,0.0,0.33,0.75,0.12,1,3
1,Regular Season - 1,167,-2,C,0.0,0.0,0.33,0.75,0.12,157,2,A,3.0,0.0,0.67,0.86,0.19,1,3
2,Regular Season - 1,158,-1,C,0.0,0.0,0.51,0.78,0.12,170,1,B,3.0,0.0,0.49,0.76,0.14,1,0
3,Regular Season - 1,170,1,B,3.0,9.0,0.49,0.76,0.14,158,-1,C,0.0,0.0,0.51,0.78,0.12,1,0
4,Regular Season - 1,159,1,B,3.0,9.0,0.49,0.80,0.10,171,-1,C,0.0,0.0,0.51,0.78,0.00,1,3


In [27]:
team_1 = pd.get_dummies(data['team_1'], prefix = 'team_1_', drop_first = True)
team_2 = pd.get_dummies(data['team_2'], prefix = 'team_2_', drop_first = True)
rating_1 = pd.get_dummies(data['rating_1'], prefix = 'team_1', drop_first = True)
rating_2 = pd.get_dummies(data['rating_2'], prefix = 'team_2', drop_first = True)

In [29]:
data_ready = (data.join(rating_1)
                  .join(rating_2)
                  .drop(['team_1', 'team_2', 'rating_1', 'rating_2', 'round'], axis=1))
data_ready.head()

,goal_difference_1,form_1,form_weighted_1,form_possession_1,form_pass_acc_1,form_shot_acc_1,goal_difference_2,form_2,form_weighted_2,form_possession_2,form_pass_acc_2,form_shot_acc_2,home_away,target,team_1_B,team_1_C,team_1_D,team_2_B,team_2_C,team_2_D
0,2,3.0,12.0,0.67,0.86,0.19,-2,0.0,0.0,0.33,0.75,0.12,1,3,0,0,0,0,1,0
1,-2,0.0,0.0,0.33,0.75,0.12,2,3.0,0.0,0.67,0.86,0.19,1,3,0,1,0,0,0,0
2,-1,0.0,0.0,0.51,0.78,0.12,1,3.0,0.0,0.49,0.76,0.14,1,0,0,1,0,1,0,0
3,1,3.0,9.0,0.49,0.76,0.14,-1,0.0,0.0,0.51,0.78,0.12,1,0,1,0,0,0,1,0
4,1,3.0,9.0,0.49,0.80,0.10,-1,0.0,0.0,0.51,0.78,0.00,1,3,1,0,0,0,1,0


## save csv files

In [30]:
data_ready.to_csv('data\\bundesliga_2018_ready.csv')
data.to_csv('data\\bundesliga_2018_full.csv')

f = open("data\\bundesliga_2018_dict.txt","w")
f.write( str(table) )
f.close()